# Hate Speech Detector - EN - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'conv'

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/Reddit/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/Reddit/clean_data.csv', 'r') as f:
        for d in list(csv.reader(f))[1:]:
            tweets.append(d[1])  # tweet
            labels.append(d[2])  # class
    with open('hsd/Reddit/perfect_data.pkl', 'wb') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Reddit/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 55132
Labels: 55132


In [5]:
list(zip(tweets[:5], labels[:5]))

[("i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
  1),
 ('This is what the left is really scared of.    https://redd.it/9rfkts', 0),
 ('That literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat?',
  0),
 ('Dumb Cunt', 1),
 ('It makes you an asshole.', 0)]

## Features extraction

In [6]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    # stop_words = set(stopwords.words('english'))
    # words = [w for w in words if not w in stop_words]
    tags = nltk.pos_tag(words)
    # words = [lemmatizer.lemmatize(w[0]) if pos_tagger(w[1]) is None else lemmatizer.lemmatize(w[0], pos_tagger(w[1])) for w in tags]
    tags = [x[1] for x in tags]
    return words, tags

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = parsed_text.strip('#')
    list_words, tag_list = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    tag_str = ' '.join(tag_list)
    return parsed_text, tag_str

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

# def get_pos_string(tweet):
#     text = preprocess(tweet)
#     tokens = word_tokenize(text)
#     tags = nltk.pos_tag(tokens)
#     tag_list = [x[1] for x in tags]
#     tag_str = ' '.join(tag_list)
    
    # return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

### Median sentences length

In [7]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence, pos_string = preprocess(d)
        # sentence = preprocess(d)
        # pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.median(wt_lengths)), int(np.median(pos_lengths))

In [8]:
opt_length, opt_wt_length, opt_pos_length = median_sentences_length(tweets)
dim = 6*20 if MODEL == 'conv' else 200

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))

Optimal all length: 19
Optimal sentence length: 19
Optimal pos sentence length: 19


### Supervised fastText wordtokens training

In [9]:
if not os.path.exists('hsd/Reddit/fasttext.ft'):
    with open('hsd/Reddit/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            text = preprocess(t)[0]
            if len(text) > 0:
                f.write('__label__{} {}\n'.format(l, text))

# load fasttext model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Reddit/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [10]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)[0]
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    #vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/medium/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.append(ft_model[w])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [11]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)
ft_model = None

In [12]:
wordtoken_features[0]

 -0.22867908, -1.0073506 ,
         0.8554437 ,  0.08904788, -1.2558832 ,  0.12104643,  0.36639142,
         0.39879748, -0.91299134, -0.04014601,  0.08627453,  0.44371277,
        -0.4338278 ,  0.9975251 ,  0.56930697, -0.3901204 , -0.13000183,
        -0.10151537, -0.53229994,  0.89158887,  0.2655875 , -0.509939  ,
        -0.1507921 , -0.70474815,  0.5499017 , -0.05195588,  1.2213051 ,
         0.83013725,  1.0016297 , -1.8891996 , -0.20725368, -1.1108195 ],
       dtype=float32),
 array([-2.15013642e-02,  4.03386503e-02,  2.43247040e-02,  1.99913643e-02,
        -1.10916365e-02, -7.83191249e-02, -7.41664544e-02,  1.51489982e-02,
        -2.46777618e-03, -2.33696662e-02, -2.71352269e-02,  5.19743524e-02,
         4.07420844e-02, -9.36147477e-03, -5.11395857e-02, -7.58136716e-03,
        -3.64146149e-03,  2.26225127e-02,  7.88684748e-03, -3.61761004e-02,
        -1.37048494e-02,  5.86107075e-02, -1.89247895e-02, -3.50688677e-03,
         4.47384492e-02, -8.71893018e-03,  5.26455194e-

### Supervised fastText pos training

In [13]:
if not os.path.exists('hsd/Reddit/fasttext_pos.ft'):
    with open('hsd/Reddit/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)[1]))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/Reddit/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [14]:
def get_pos_fts(data, length):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = preprocess(d)[1]
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    pos_tags = [pad_words(pt, length) for pt in pos_tags]
    
    ft_matrices = []
    for pt in pos_tags:
        ft_matrix = []
        for t in pt:
            ft_matrix.append(ft_pos_model[t])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [15]:
pos_features = get_pos_fts(tweets, opt_pos_length)
tweets = None
ft_pos_model = None

In [16]:
pos_features[0]

68e-02, -3.34299207e-02,
        -4.48512249e-02,  8.58354643e-02, -1.06395478e-03, -6.09381869e-02,
        -9.29926932e-02,  9.26140919e-02, -3.63640562e-02, -7.89020956e-02,
         6.23624865e-03, -1.32546112e-01,  5.53525724e-02,  8.37932825e-02,
         1.49103865e-01,  3.63024771e-02, -1.00650795e-01, -1.76700100e-01,
        -7.60897920e-02,  6.43940493e-02, -5.36994785e-02, -5.52786179e-02,
        -2.21901983e-02, -5.39563922e-03,  8.24100897e-02, -4.65049706e-02,
         6.59383908e-02,  7.19505399e-02, -9.33090597e-02,  4.07069288e-02,
         1.75074879e-02, -1.07715704e-01, -1.71629861e-02,  2.73557752e-02],
       dtype=float32),
 array([-0.20734665,  0.4355399 ,  0.20166968,  0.1855236 , -0.05438184,
        -0.24447659,  0.13261203, -0.00922825,  0.12079176,  0.1248029 ,
        -0.09909526,  0.11192505,  0.09976787, -0.0747248 , -0.5135353 ,
         0.08995245, -0.03938875,  0.01320488, -0.00760706, -0.01048971,
         0.0919881 ,  0.0448121 , -0.00387972, -0.0

In [17]:
np.array(wordtoken_features)

array([[[ 0.03673194, -0.06451502, -0.04818777, ..., -0.16086693,
         -0.02701685, -0.08516414],
        [-0.02415899,  0.04267944,  0.025993  , ...,  0.06191521,
         -0.00174052,  0.03179947],
        [-0.08524284,  0.17059486,  0.09587625, ...,  0.3041841 ,
          0.03544652,  0.18275881],
        ...,
        [ 0.03322716, -0.0812467 , -0.04391546, ..., -0.14735347,
         -0.02358901, -0.07964789],
        [ 0.00305167, -0.00235601, -0.00560494, ...,  0.00038792,
          0.00793241,  0.00042148],
        [-0.04192798,  0.08787871,  0.04221893, ...,  0.12818556,
          0.00797077,  0.09204595]],

       [[ 0.03410764, -0.08713895, -0.04500911, ..., -0.15183271,
         -0.00927543, -0.09204976],
        [-0.05981733,  0.04689059,  0.03143451, ...,  0.02779103,
         -0.00436429,  0.01408869],
        [-0.04032508,  0.06487818,  0.04369756, ...,  0.11405523,
          0.01720574,  0.06714447],
        ...,
        [ 0.00487847, -0.00895385, -0.01618856, ..., -

In [18]:
words_token_shape = np.array(wordtoken_features).shape[1]

In [19]:
np.array(pos_features).shape

(55132, 19, 120)

### All features

In [20]:
# #Now join them all up
#features = np.array(wordtoken_features)
features = np.concatenate([wordtoken_features, pos_features], axis=1)
wordtoken_features = None
pos_features = None

In [21]:
features.shape

(55132, 38, 120)

In [22]:
features[0]


array([[ 0.03673194, -0.06451502, -0.04818777, ..., -0.16086693,
        -0.02701685, -0.08516414],
       [-0.02415899,  0.04267944,  0.025993  , ...,  0.06191521,
        -0.00174052,  0.03179947],
       [-0.08524284,  0.17059486,  0.09587625, ...,  0.3041841 ,
         0.03544652,  0.18275881],
       ...,
       [ 0.05145825,  0.3328534 , -0.00502174, ...,  0.32843122,
         0.33541927,  0.1757834 ],
       [ 0.36963922, -0.176161  , -0.04095278, ..., -0.1077157 ,
        -0.01716299,  0.02735578],
       [-0.21861666,  0.01753578,  0.07147932, ...,  0.09935798,
         0.20915161,  0.0636612 ]], dtype=float32)

## Save features & labels

In [27]:
archive = dir_archive('hsd/Reddit/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': words_token_shape}, serialized=True)
archive.dump()
del archive